# Lab 3: EOG 
This notebook goes through processing EOG Signals using NeuroKit.

In [ ]:
# Run only once
!pip3 install neurokit2
!pip3 install mne
!pip3 install sklearn

In [ ]:
import pandas as pd
import scipy
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np
import neurokit2 as nk
import os
plt.rcParams['figure.figsize'] = [15, 9]  # Larger images

In [ ]:
import pyxdf

def xdf_to_dataframe(xdf_data):
    ''' Xdf Data should be a list of streams (dictionaries)
        Function returns a dictionary of dataframes, one dataframe per stream'''  
    dataframes = {}
    for stream in xdf_data:
        df = pd.DataFrame()
        data = stream['time_series']
        timestamps = stream['time_stamps']
        df['Time'] = timestamps
        chan_names, units = get_channel_names(stream['info'])
        counts = data.shape[0]
        for series, name, unit in zip(range(data.shape[1]), chan_names, units):
            df[name[0]]  = data[:, series]
            if unit:
                df[name[0] + '_Unit'] = np.repeat(unit, counts)
        
        for item in stream['info']:
            if item not in ['name', 'desc', 'data']:
                try:
                    df[item] = np.repeat(stream['info'][item], counts)
                except:
                    continue
        dataframes[stream['info']['name'][0]] = df
        
    return dataframes
            
        
        

def get_channel_names(info):
    channels = info['desc'][0]['channels'][0]['channel']
    names = [chan['label'] for chan in channels ]
    units = [chan['unit'] for chan in channels ]
    return names, units

# EOG Procesing
We are first going to read in raw EOG data from one lab member

In [ ]:
directory = 'lab data/lab 3/lab 3 EOGs'
participant= 'PARTICPANT'
file = 'FILE'
data, header = pyxdf.load_xdf(os.path.join(directory, participant, file))
dfs = xdf_to_dataframe(data)
display(dfs['BioRadio-20314'])
eogdf = dfs['BioRadio-20314']

In [ ]:
'''Plot the raw signals'''
nk.signal_plot(eogdf[['CH1', 'CH2']], sampling_rate=256)

In [ ]:
'''Plot the raw and clean signals'''
eogdf['CH1_Cleaned'] = nk.eog_clean(eogdf['CH1'], sampling_rate=256, method='neurokit')
nk.signal_plot([eogdf['CH1'], eogdf['CH1_Cleaned']], labels=["Raw Signal", "Cleaned Signal"])
plt.figure()
eogdf['CH2_Cleaned'] = nk.eog_clean(eogdf['CH2'], sampling_rate=256, method='neurokit')
nk.signal_plot([eogdf['CH2'], eogdf['CH2_Cleaned']], labels=["Raw Signal", "Cleaned Signal"])
plt.figure()
nk.signal_plot([eogdf['CH1_Cleaned'], eogdf['CH2_Cleaned']], labels=['CH1', 'CH2'])

# Detect and Find Blinks
We can run a peak detection algorithm to find blinks

In [ ]:
blinks = nk.eog_findpeaks(eogdf['CH1_Cleaned'], sampling_rate=256, method="mne")
blinks

# Feature Extraction

We are going to extract features according to this [paper](https://www.sciencedirect.com/science/article/pii/S1877705812012957])

Specifically:
1. Maximum Peak Value for V and H (CH 1 and 2) 
- Hint: Max function
2. Minimum Peak Value for V and H 
- Hint: Min function
3. Maximum Peak Amplitude Position for V and H
- Hint: nk.eog_findpeaks and choose the first non-zero value
4. Area under Curve for V and H
- Hint: Take the absolute value of each channel and take the sum of the vector ($\Sigma_{i=1}^N \vert x \vert$)
5. Variance for V and H
- Hint: np.var

This should result in 10 features for each sample.

## Fill in the 5 feature methods below

In [ ]:
def extract_features(dataframe, channels):
    '''Cycle through each channel'''
    feat_dict = {}
    for chan in channels:
        feat_dict[chan + 'PAV'] = get_max_peak(dataframe[chan])
        feat_dict[chan + 'VAV'] = get_min_peak(dataframe[chan])
        feat_dict[chan + 'PAP'] = get_peak_amplitude_position(dataframe[chan])
        feat_dict[chan + 'AUC'] = get_AUC([chan])
        feat_dict[chan + 'VAR'] = get_Variance(dataframe[chan])
    return feat_dict
        

def get_max_peak(signal):
    '''Your code here'''
    return 1
    
def get_min_peak(signal):
    '''Your code here'''
    return 1
    
def get_peak_amplitude_position(signal):
    '''Your Code here'''
    return 1
    
def get_AUC(signal):
    '''Your code here'''
    return 1
    
def get_Variance(signal):
    '''Your code here'''
    return 1
        



In [ ]:
'''Test your code here'''
feats = extract_features(eogdf, ['CH1_Cleaned', 'CH2_Cleaned'])
display(feats)
'''This should display a dictionary with your extracted features'''

# Generate a Feature Dataframe
Now we are going to build our feature dataframe so we can learn off of it

You may have to change the process a bit to match your naming conventions. I am assuming you have the following structure:

Data -> Participant Data -> .xdf files for each class

the .xdf files is assumed to be named CLASSNameTRIAL#.xdf (i.e., lookDown1.xdf, lookDown2.xdf)

In [ ]:
directory    = 'lab data/lab 3/lab 3 EOGs'
participants = ['P1', 'P2']
classes = ['lookDown', 'lookLeft', 'lookRight', 'lookUp']
sensor = 'BioRadio-20314'


feature_df = pd.DataFrame()

'''Cycle through participants, class names, and trials'''
for participant in participants:
    for cls in classes:
        for trial in range(5):
            data, header = pyxdf.load_xdf(os.path.join(directory, participant, cls + str(trial + 1) + '.xdf'))
            df = xdf_to_dataframe(data)[sensor]
            df['CH1'] = nk.eog_clean(df['CH1'], sampling_rate=256, method='neurokit')
            df['CH2'] = nk.eog_clean(df['CH2'], sampling_rate=256, method='neurokit')
            feats = extract_features(df, ['CH1', 'CH2'])
            feats['Participant'] = participant
            feats['Class'] = cls
            feats['Trial'] = str(trial + 1)
            feature_df = pd.concat([feature_df, pd.DataFrame(feats, index=[0])], ignore_index=True)
display(feature_df)
            
'''For three participants, you should have ~60 samples'''


## Examine Data by Class

In the following cells, you should do the following:
1. Group by class
2. Examine the descripitive statistics for each feature by class
3. Examine the correlations by class (look at pandas .corr() method)

# SVM Machine Learning
Now we are going to do some machine learning with SVMs

In [ ]:
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA

In [ ]:
'''We are going to do Leave-One-Out Cross Validation'''
groups = feature_df['Participant'].values # Specify groups
X = feature_df.drop(columns=['Trial', 'Class', 'Participant']).values # specify Feature columns

'''Create our Labels from Categories'''
le = LabelEncoder()
le.fit(feature_df['Class'].values) # Specify Classes
y = le.transform(feature_df['Class'].values) # Transform categories into class labels
display(y)

''' Initialize groups'''
logo = LeaveOneGroupOut()
'''Look at how many groups we have (should be the number of participants)'''
logo.get_n_splits(X, y, groups)




In [ ]:
predicted = np.array([])
true = np.array([])

'''We can create a classifaction pipeline'''
clf = make_pipeline(StandardScaler(), # Standardize the inputs
                    PCA(n_components=0.9, svd_solver='full'),  # PCA with 0.9 as the threshold for explain variance
                    SVC(kernel='rbf', gamma='auto', class_weight='balanced')) # SVM with RBF kernel

''' We can cycle through each group, fit our model, and record our results'''
for train_index, test_index in logo.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index] # get x training/testing
    y_train, y_test = y[train_index], y[test_index] # get y training/testing
    clf.fit(X_train, y_train) # fit our model
    predicted = np.hstack((predicted, clf.predict(X_test)))
    true = np.hstack((true, y_test))
    

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
'''View Results'''
display(pd.DataFrame(classification_report(true, predicted,output_dict=True)).transpose())

cm = confusion_matrix(true, predicted)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

# Your turn!
For your lab report, answer the following:

1. What features were the most correlated with each other?
2. Above we used a SVM with a RBF kernel, what are the accuracies using a polynomial and linear kernel?
3. What is a SKLearn pipeline and why is it useful?
4. Was there a participant that the model performed worse than the other participants? If so, explain why.